In [4]:
!pip uninstall -y pyarrow datasets
!pip install --no-use-pep517 pyarrow
!pip install datasets
!pip install accelerate -U

Found existing installation: pyarrow 16.0.0
Uninstalling pyarrow-16.0.0:
  Successfully uninstalled pyarrow-16.0.0
Found existing installation: datasets 2.19.0
Uninstalling datasets-2.19.0:
  Successfully uninstalled datasets-2.19.0
  Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (40.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.11.3 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.0.0 which is incompatible.
  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.0-py3-none-any.whl (542 kB)


In [1]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report


dataset = load_dataset("Hello-SimpleAI/HC3", name="all")

# Create lists from the dataset
human_answers = [string for sublist in dataset['train']['human_answers'] for string in sublist if sublist]
chatgpt_answers = [string for sublist in dataset['train']['chatgpt_answers'] for string in sublist if sublist]

# Create a Dataset from the lists
data_dict = {
    "texts": human_answers + chatgpt_answers,
    "labels": [0] * len(human_answers) + [1] * len(chatgpt_answers)
}

hf_dataset = Dataset.from_dict(data_dict)


In [2]:
# Retrieve the texts and labels from the dataset.
texts = hf_dataset["texts"]
labels = hf_dataset["labels"]

# Vectorize the texts, which converts the text data into numerical feature vectors.
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, np.array(labels), test_size=0.2, random_state=42)

# Create and train the Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Model Evaluation
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle Naïf Bayésien : {:.2f}%".format(accuracy * 100))


Précision du modèle Naïf Bayésien : 90.49%


In [4]:
# Predict on set test
y_pred = clf.predict(X_test)

# Print the confusion matrix and the classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Matrice de confusion:\n", conf_matrix)
print("\nRapport de classification:\n", class_report)


Matrice de confusion:
 [[10897   855]
 [  770  4568]]

Rapport de classification:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     11752
           1       0.84      0.86      0.85      5338

    accuracy                           0.90     17090
   macro avg       0.89      0.89      0.89     17090
weighted avg       0.91      0.90      0.91     17090

